In [ ]:
import logging
from dataclasses import dataclass
from typing import TypeVar

from osmocom.tlv import BER_TLV_IE
from osmocom.utils import b2h, h2b
from pySim.commands import SimCardCommands
from pySim.euicc import (
    EuiccConfiguredAddresses,
    EuiccInfo2,
    GetEuiccChallenge,
    GetEuiccData,
    TagList,
)
from pySim.transport.pcsc import PcscSimLink

T = TypeVar("T")


class ESIMException(Exception):
    def __init__(self, sw: str):
        self.sw = sw

    def __str__(self):
        return f"Error: {self.sw}"


class APDUPacket:
    def __init__(
        self,
        cla: int,
        ins: int,
        p1: int,
        p2: int,
        data: bytes = b"",
        le: int = 0,
    ):
        """Initializes an APDU packet."""
        self.cla = cla
        self.ins = ins
        self.p1 = p1
        self.p2 = p2
        self.data = data
        self.le = le

    @property
    def lc(self):
        return len(self.data) if self.data else 0

    def to_hex(self) -> str:
        """Returns the APDU packet as a hex string."""
        apdu = bytearray([self.cla, self.ins, self.p1, self.p2])
        if self.lc > 0:
            apdu.append(self.lc)
            apdu.extend(self.data)
        if self.le > 0:
            apdu.append(self.le)

        return apdu.hex()


class Application:
    aid: str
    alternative_aids: list[str] = []
    name: str

    def __init__(self, link: PcscSimLink, aid: str | None = None):
        self.link = link
        self.aid = aid or self.aid

    def send_apdu(self, apdu: APDUPacket, expected_sw: int = 0x9000):
        data, sw = self.link.send_apdu(apdu.to_hex())
        if sw != expected_sw:
            raise Exception(f"Error: {sw}")

        return data

    def merge_dicts(self, dicts: list[dict]) -> dict:
        return {k: v for d in dicts for k, v in d.items()}

    def store_data(self, data: str):
        apdu = APDUPacket(cla=0x80, ins=0xE2, p1=0x91, p2=0x00, data=h2b(data))
        return self.link.send_apdu_checksw(apdu.to_hex(), sw="9000")

    def store_data_tlv(
        self,
        command_cls: type[BER_TLV_IE],
        response_cls: type[BER_TLV_IE] | None = None,
    ) -> type[BER_TLV_IE] | None:
        command_encoded = command_cls().to_tlv()

        if len(command_encoded) > 255:
            raise ValueError("Data too long")

        data, _ = self.store_data(b2h(command_encoded))

        if response_cls is None:
            response_cls = command_cls

        if data:
            response = response_cls()
            response.from_tlv(h2b(data))
            return response

        return None


@dataclass
class Profile:
    smdpp_address: str
    matching_id: str
    confirmation_code: str | None = None

    def full_activation_code(self) -> str:
        return f"LPA:1${self.smdpp_address}${self.matching_id}"

    @staticmethod
    def from_activation_code(activation_code: str) -> "Profile":
        if not activation_code.startswith("LPA:1$"):
            raise ValueError("Invalid activation code")

        parts = activation_code.split("$")
        assert len(parts) == 3

        return Profile(parts[1], parts[2])


class USIM(Application):
    aid = "a0000000871002"
    name = "USIM"


class ISDR(Application):
    aid = "A0000005591010FFFFFFFF8900000100"
    name = "ISDR"

    def get_euicc_challenge(self) -> str:
        euicc_challenge = self.store_data_tlv(GetEuiccChallenge)
        return euicc_challenge.to_dict().get("get_euicc_challenge")[0][
            "euicc_challenge"
        ]

    def get_euicc_info_2(self) -> list[dict]:
        command = self.store_data_tlv(EuiccInfo2)
        euicc_info = command.to_dict().get("euicc_info2")
        return self.merge_dicts(euicc_info)

    def get_configured_addresses(self) -> str:
        command = self.store_data_tlv(EuiccConfiguredAddresses)
        addresses = command.to_dict().get("euicc_configured_addresses")
        return self.merge_dicts(addresses)

    def get_eid(self) -> str:
        command = GetEuiccData(children=[TagList(decoded=[0x5A])])
        eid_data = self.store_data_tlv(command, GetEuiccData)
        return eid_data.to_dict().get("get_euicc_data")[0]["eid_value"]

    def download_profile(self, profile: Profile):
        smdpp_address = profile.smdpp_address
        if not smdpp_address:
            smdpp_address = self.get_configured_addresses().get("smdpp_address")

        logging.info(f"Downloading profile from {smdpp_address}")
        challenge = self.get_euicc_challenge()


class ECDSA(Application):
    aid = "A0000005591010FFFFFFFF8900000200"
    name = "ECDSA"


class ISDP(Application):
    aid = "A0000005591010FFFFFFFF8900000300"
    name = "ISDP"


class Card:
    supported_applications: dict[type[Application], Application] = {}

    def __init__(self, link: PcscSimLink):
        self.link = link
        applications: list[Application] = [USIM, ISDR, ECDSA, ISDP]
        self.commands = SimCardCommands(self.link)
        self.commands.cla_byte = "00"
        self.executed_commands = []

        for application in applications:
            for aid in [application.aid] + application.alternative_aids:
                try:
                    self.commands.select_adf(aid)
                    self.supported_applications[application] = application(
                        self.link, aid
                    )
                    break
                except Exception:
                    pass

    def __getattribute__(self, name):
        attr = object.__getattribute__(self, name)
        if hasattr(attr, "__call__"):

            def trace_function(*args, **kwargs):
                logging.debug("calling %s" % attr.__name__)
                result = attr(*args, **kwargs)
                logging.debug("done calling %s" % attr.__name__)
                return result

            return trace_function
        else:
            return attr

    @property
    def isd_r(self) -> ISDR:
        isd_r = self.supported_applications.get(ISDR)
        if not isd_r:
            raise Exception("ISDR application not supported")

        self.commands.select_adf(ISDR.aid)
        return isd_r

In [22]:
with PcscSimLink() as link:
    card = Card(link)
    print([app.name for app in card.supported_applications.values()])
    print(card.isd_r.get_euicc_info_2())
    print(card.isd_r.get_euicc_challenge())
    print(card.isd_r.get_configured_addresses())

['USIM', 'ISDR', 'ECDSA']
{'profile_version': '2.3.1', 'svn': '2.3.0', 'euicc_firmware_ver': '36.17.4', 'ext_card_resource': '81010b82040017520083027fe3', 'uicc_capability': '067f36f7c0', 'ts102241_version': '15.1.0', 'global_platform_version': '2.3.0', 'rsp_capability': '0398', 'euicc_ci_pki_list_for_verification': [{'subject_key_identifier': '81370f5125d0b1d408d4c3b232e6d25e795bebfb'}], 'euicc_ci_pki_list_for_signing': [{'subject_key_identifier': '81370f5125d0b1d408d4c3b232e6d25e795bebfb'}], 'pp_version': '1.0.0', 'ss_acreditation_number': 'KN-DN-UP-0924'}
9e88e55939df4037499e76f2ffa17464
{'default_dp_address': 'smdp-plus-0.eu.cd.rsp.kigen.com', 'root_ds_address': 'lpa.ds.gsma.com'}


In [3]:
from pySim.apdu import Apdu

APDU = Apdu("80E2910003bf2200")
print(APDU.cla, str(APDU.ins), str(APDU.p1), str(APDU.p2), str(APDU.lc))

128 226 145 0 3
